In [2]:
import sys
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Datasets')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines')

In [3]:
from pascalpart import get_pascalpart_masks
from utils import read_txt_file
from tqdm import tqdm
import os
from PIL import Image
import numpy as np

In [3]:
pascal_image_dir = "/data/Pascal_VOC_2012/VOCdevkit/VOC2012/JPEGImages" # 17125 images
annotations_path= "/data/PartSegmentationDatasets/PascalPart/Annotations_Part"
val_filepath = "/data/PartSegmentationDatasets/PascalPart/val.txt" # 925 images. File contains just the file prefix. Add .jpg extension for images, and .mat extension for annotations
base_pred_dir = "/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM"

In [13]:
# Uncomment the following if you want to evaluate on all images
# val_filenames = read_txt_file(val_filepath)
# req_filenames = val_filenames

# Evaluate only on existing results 
req_filenames = os.listdir(base_pred_dir)

In [15]:
def combine_masks(masks):
    combined_mask = masks[0]
    for i in range(1, len(masks)):
        combined_mask = combined_mask | masks[i]
    return combined_mask

In [16]:
from chainercv.evaluations import calc_semantic_segmentation_confusion
def compute_miou(preds, gts):
    '''
    This function takes two lists as inputs, where:
    preds: [list of 2-D prediction where each pixel corresponds to pixel-label]
    gts: [list of ground-truths in 2-D space, contains -1 value for critical pixels]
    
    returns mean of the iou for all classes
    '''
    confusion = calc_semantic_segmentation_confusion(preds, gts)

    gtj = confusion.sum(axis=1)
    resj = confusion.sum(axis=0)
    gtjresj = np.diag(confusion)
    denominator = gtj + resj - gtjresj
    iou = gtjresj / denominator

    return iou, confusion

In [19]:
def get_glamm_predicted_mask(base_pred_dir, filename_prefix, class_name, is_crop=False):
    """
        Get the predicted mask for a given image and class from GLAMM predictions.
        Args:
            base_pred_dir (str): Base directory where GLAMM predictions are stored.
            filename_prefix (str): Prefix of the filename (without extension).
            class_name (str): Name of the class for which the mask is required.
    """
    class_name = class_name.replace(" ", "_")
    # all pred masks for this filename is in a directory named filename_prefix
    pred_masks_dir = os.path.join(base_pred_dir, filename_prefix)
    # the required mask is in a file named class_name.png. here, class_name can either be the object class (like person) or the object || part class (like person_head)
    if not is_crop:
        req_pred_mask = os.path.join(pred_masks_dir, class_name + ".png")
    else:
        req_pred_mask = os.path.join(pred_masks_dir, class_name+'_crop' + ".png")
    # Load the mask
    mask = Image.open(req_pred_mask).convert("L")  # grayscale

    # Convert to numpy, then 0/1
    mask_np = np.array(mask)
    binary_mask = (mask_np > 0).astype(np.uint8)
    
    return binary_mask


In [ ]:
# comment out the object related code if you want to compute only part ious
# obj_ious = []
part_ious = []
count_no_obj = 0
for filename in tqdm(req_filenames):
    # gt_obj_masks = []
    # pred_obj_masks = []

    gt_part_masks = []
    pred_part_masks = []

    img_filepath = os.path.join(pascal_image_dir, filename+'.jpg')

    annot_filename = filename + '.mat'
    anno_dict = get_pascalpart_masks(annot_filename, annotations_path, images_path=pascal_image_dir)

    for obj_name, anno in anno_dict.items():
        # obj_masks = anno['object_maps']
        # obj_mask = combine_masks(obj_masks)
        # gt_obj_masks.append(obj_mask)
        # pred_obj_mask = get_glamm_predicted_mask(base_pred_dir, filename, obj_name)
        # pred_obj_masks.append(pred_obj_mask)

        parts_masks = anno['parts']
        for part_name, masks in parts_masks.items():
            part_mask = combine_masks(masks)
            gt_part_masks.append(part_mask)
            part_full_name = obj_name + "_" + part_name
            pred_part_mask = get_glamm_predicted_mask(base_pred_dir, filename, part_full_name, is_crop=True)
            pred_part_masks.append(pred_part_mask)

    if len(pred_part_masks) == 0:
        # no part present in this image
        print(f'no parts present in {filename}, skipping...')
        count_no_obj += 1
        continue
    
    # obj_iou_w_bg, obj_confusion = compute_miou(pred_obj_masks, gt_obj_masks)
    # obj_ious.append(obj_iou_w_bg[1]) # only taking the iou of the object class, not the background


    part_iou_w_bg, part_confusion = compute_miou(pred_part_masks, gt_part_masks)
    part_ious.append(part_iou_w_bg[1]) # only taking the iou of the part class, not the background
    

  0%|          | 0/83 [00:00<?, ?it/s]


ValueError: Shape of ground truth and prediction should be same.

In [48]:
np.mean(obj_ious)

0.7910982110167357

In [49]:
np.mean(part_ious)

0.22074454816350528